In [ ]:
# Notebook to do a zeroshot analysis on our big-dataset of frames and say % each matches SF along with confidence %
# (This new dataset will later be probed)

# pip install -U pandas together

import pandas as pd
import numpy as np
import re

df2 = pd.read_csv("../1-Generation-and-Dataset/data-generated-stories-frames-202505.csv")

for ix, row in df2.sort_values(by='Frame', ascending=True).iterrows():
    item = {}
    item['story'] = row['StorySanit'] if row['StorySanit'] else row['Story']  # if has sanitized version, use that one
    item['frame_label'] = row['Frame']  # already filtered to only score=1s, btw
    item['source_notes'] = row['Model'] + '-' + row['Source'] + (' (halu)' if row['Hallucination']=='1' else '')
    item['humdoubt'] = row['AnnotationScore'] != 1
    mask = df2['Story'].str.contains(re.escape(item['story']))
    item['L3_SF'] = row['L3_SF'] if 'L3_SF' in row else None    
    item['L3_NP'] = row['L3_NP'] if 'L3_NP' in row else None
    item['L3_top5'] = row['L3_top5'] if 'L3_top5' in row else None  
    item['L3s_SF'] = row['L3s_SF'] if 'L3s_SF' in row else None
    item['L3s_NP'] = row['L3s_NP'] if 'L3s_NP' in row else None
    item['L2_SF'] = row['L2_SF'] if 'L2_SF' in row else None    
    item['L2_NP'] = row['L2_NP'] if 'L2_NP' in row else None
    df2 = pd.concat([df2, pd.DataFrame([item])], ignore_index=True)


        

df2 shape:  <bound method NDFrame.head of        ID    Model Source                                        Frame  \
0       0   llama2   orig                                strict-father   
1       2   llama2  scifi                                strict-father   
2       3   llama2   orig                             nurturing-parent   
3       5   llama2  scifi                             nurturing-parent   
4       6   llama2   orig                                  us-vs.-them   
..    ...      ...    ...                                          ...   
265  1000   vicuna   orig             information-spreads-like-a-virus   
266  1001  mistral   orig             information-spreads-like-a-virus   
267  1002   vicuna  bible                             nurturing-parent   
268  1003     gpt4  bible             information-spreads-like-a-virus   
269  1004       yi   orig  information-follows-individual-dispositions   

                                                 Story  \
0    The st

In [9]:
import pandas as pd
import numpy as np

from together import Together 
client = Together(api_key="68d210401de822340a017cbfa0486b12a3d14a12cc7f7c7c247844fc51edbbd9")      

df2['L3_SF_'] = np.nan
df2['L3_NP_'] = np.nan


In [10]:
import re

for ix, row in df2.sort_values(by='story', ascending=True).iterrows():
    #if row['humdoubt'] is True:
        # already done!
    #    continue

    print(ix, row['frame_label'], end=" → ")

    # TODO: REDO LLAMA3-70B once more with `EVOKE`` INSTEAD OF `INVOKE` to see if results change? -- ALSO REDO 'TEXT' INSTEAD OF 'STORY'

    # # L2: SF percent
    # prompt = 'What percentage does the following text invoke the "strict father" frame? (Please give just the percentage with no additional words)\n'
    # prompt += '``' + row['story'] + '``'
    # response = client.chat.completions.create(
    #     model="meta-llama/LLama-2-7b-chat-hf",  
    #     messages=[{"role": "user",  "content": prompt}],
    #     temperature=0,   # hmmm
    #     max_tokens=50
    # )
    # resp_text = response.choices[0].message.content
    # resp_text = resp_text.lower().replace("\n", " ").strip()  
    # pattern = r'(?:strict father.*?(\d+)%|(\d+)%$)'
    # match = re.search(pattern, resp_text)
    # df2.loc[ix, 'L2_SF'] = int(match.group(1) or match.group(2)) if match else resp_text.replace('%', '')

    # #L2: NP percent
    # prompt = 'What percentage does the following text invoke the "nuturing parent" frame? (Please give just the percentage with no additional words)\n'
    # prompt += '``' + row['story'] + '``'
    # response = client.chat.completions.create(
    #     model="meta-llama/LLama-2-7b-chat-hf",  
    #     messages=[{"role": "user",  "content": prompt}],
    #     temperature=0,   # hmmm
    #     max_tokens=50
    # )
    # resp_text = response.choices[0].message.content
    # resp_text = resp_text.lower().replace("\n", " ").strip()  
    # pattern = r'(?:nurturing parent.*?(\d+)%|(\d+)%$)'
    # match = re.search(pattern, resp_text)
    # df2.loc[ix, 'L2_NP'] = int(match.group(1) or match.group(2)) if match else resp_text

    # L3: SF percent
    prompt = 'What percentage does the following text evoke the "strict father" frame? (Please give just the percentage with no additional words)\n'
    prompt += '``' + row['story'] + '``'
    response = client.chat.completions.create(
        model= "meta-llama/Llama-3-8b-chat-hf",   # "meta-llama/Llama-3-8b-chat-hf",  #"meta-llama/Llama-3-70b-chat-hf"
        messages=[{"role": "user",  "content": prompt}],
        temperature=0,   # hmmm
        max_tokens=50
    )
    resp_text = response.choices[0].message.content
    resp_text = resp_text.lower().replace("\n", " ").replace("sure, here are my estimates:", "").replace("sure! here are my estimates:", "").replace("here are the percentages:", "").replace(" frame", "").replace("* ", "").replace(" level", "").replace('"', '').replace(',', '')
    df2.loc[ix, 'L3_SF_'] = int(resp_text.strip().replace('%', ''))

    # # L3: NP percent
    prompt = 'What percentage does the following text evoke the "nuturing parent" frame? (Please give just the percentage with no additional words)\n'
    prompt += '``' + row['story'] + '``'
    response = client.chat.completions.create(
        model="meta-llama/Llama-3-8b-chat-hf",   # "meta-llama/Llama-3-8b-chat-hf",  # "meta-llama/Llama-3-70b-chat-hf"
        messages=[{"role": "user",  "content": prompt}],
        temperature=0,   # hmmm
        max_tokens=50
    )
    resp_text = response.choices[0].message.content
    resp_text = resp_text.lower().replace("\n", " ").replace("sure, here are my estimates:", "").replace("sure! here are my estimates:", "").replace("here are the percentages:", "").replace(" frame", "").replace("* ", "").replace(" level", "").replace('"', '').replace(',', '')
    df2.loc[ix, 'L3_NP_'] = resp_text.strip().replace('%', '')

    #    # L3: SF percent
    # prompt = 'What percentage does the following text invoke the "strict father" frame? (Please give just the percentage with no additional words)\n'
    # prompt += '``' + row['story'] + '``'
    # response = client.chat.completions.create(
    #     model="meta-llama/Llama-3-70b-chat-hf",   # 
    #     messages=[{"role": "user",  "content": prompt}],
    #     temperature=0,   # hmmm
    #     max_tokens=50
    # )
    # resp_text = response.choices[0].message.content
    # resp_text = resp_text.lower().replace("\n", " ").replace("sure, here are my estimates:", "").replace("sure! here are my estimates:", "").replace("here are the percentages:", "").replace(" frame", "").replace("* ", "").replace(" level", "").replace('"', '').replace(',', '')
    # df2.loc[ix, 'L3_SF'] = resp_text.strip().replace('%', '')

    # # # L3: NP percent
    # prompt = 'What percentage does the following text invoke the "nuturing parent" frame? (Please give just the percentage with no additional words)\n'
    # prompt += '``' + row['story'] + '``'
    # response = client.chat.completions.create(
    #     model="meta-llama/Llama-3-70b-chat-hf",   # 
    #     messages=[{"role": "user",  "content": prompt}],
    #     temperature=0,   # hmmm
    #     max_tokens=50
    # )
    # resp_text = response.choices[0].message.content
    # resp_text = resp_text.lower().replace("\n", " ").replace("sure, here are my estimates:", "").replace("sure! here are my estimates:", "").replace("here are the percentages:", "").replace(" frame", "").replace("* ", "").replace(" level", "").replace('"', '').replace(',', '')
    # df2.loc[ix, 'L3_NP'] = resp_text.strip().replace('%', '')

    # # L3: top five frames
    # prompt = 'Can you tell me which major cognitive frames are invoked by the following text? (Please keep your answer strictly short and name max 5 frames with no explanation)\n'
    # prompt += '``' + row['story'] + '``'
    # response = client.chat.completions.create(
    #     model="meta-llama/Llama-3-70b-chat-hf",   # "meta-llama/Llama-3-8b-chat-hf",
    #     messages=[{"role": "user",  "content": prompt}],
    #     temperature=0,   # hmmm
    #     max_tokens=50
    # )
    # resp_text = response.choices[0].message.content
    # resp_text = resp_text.replace("\n", " ").replace("The frame invoked by", "").replace("The answer is:", "").replace("Sure!", "").replace(" The story invokes the", "")
    # resp_text = resp_text.replace("Here are the major cognitive frames invoked by the story:", "").replace("Here are 5 major cognitive frames invoked by the story:", "").replace("Here are the 5 major cognitive frames invoked by the passage:", "").replace("Here are the 5 major cognitive frames invoked by the story:", "")
    # resp_text = resp_text.replace("1. ", "").replace(" 2.", ",").replace(" 3.", ",").replace(" 4.", ",").replace(" 5.", ",")
    # df2.loc[ix, 'L3_top5'] = resp_text.strip()

    print(df2.loc[ix, 'L3_SF'], df2.loc[ix, 'L3_NP'], "→", df2.loc[ix, 'L3_SF_'], df2.loc[ix, 'L3_NP_'])



290 strict-father → 

/tmp/ipykernel_986/1756133261.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '34' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[ix, 'L3_NP_'] = resp_text.strip().replace('%', '')


nan nan → 85.0 34
323 we-are-all-in-this-together → nan nan → 12.0 34
279 nurturing-parent → nan nan → 12.0 85
312 we-are-all-in-this-together → nan nan → 14.0 34
280 nurturing-parent → nan nan → 0.0 82
273 nurturing-parent → nan nan → 0.0 80
271 nurturing-parent → nan nan → 12.0 72
284 nurturing-parent → nan nan → 80.0 80
304 strict-father → nan nan → 85.0 75
325 we-are-all-in-this-together → None None → 23.0 34
317 we-are-all-in-this-together → nan nan → 34.0 63
305 strict-father → nan nan → 85.0 75
274 nurturing-parent → nan nan → 12.0 82
283 nurturing-parent → nan nan → 0.0 85
318 we-are-all-in-this-together → nan nan → 23.0 63
311 we-are-all-in-this-together → nan nan → 34.0 63
270 nurturing-parent → nan nan → 0.0 85
287 nurturing-parent → nan nan → 34.0 85
292 strict-father → nan nan → 34.0 34
288 nurturing-parent → nan nan → 12.0 63
302 strict-father → nan nan → 85.0 34
285 nurturing-parent → nan nan → 63.0 63
316 we-are-all-in-this-together → nan nan → 34.0 63
275 nurturing-par

KeyboardInterrupt: 